## Prepare dataset

### Option 1: Connect to Bucket on CHI@TACC
First, create the volume:

``` bash
# run on node
curl https://rclone.org/install.sh | sudo bash

# run on node-persist
# this line makes sure user_allow_other is un-commented in /etc/fuse.conf
sudo sed -i '/^#user_allow_other/s/^#//' /etc/fuse.conf

# run on node-persist
mkdir -p ~/.config/rclone
nano  ~/.config/rclone/rclone.conf
```

Paste the following
``` bash
[chi_tacc]
type = swift
user_id = 4376b58eeabc6e4ec2191921c0a2b07d8f245ba944cb9cd7d21e299a67b25358
application_credential_id = 7f5aa830dd8743659ce482a802e3d832
application_credential_secret = WENItrG3L0i9y-OPEmPj1GG6HXnWNnvUaq8v4nhM3ttUdoeMEVVa5o4acsCI8JYhmiyvkr0nacmEwWlRk2BkWg
auth = https://chi.tacc.chameleoncloud.org:5000/v3
region = CHI@TACC
```

Then test it, run
``` bash
# run on node-persist
rclone lsd chi_tacc:
```

The next step is to create a mount point for the data in the local filesystem:
``` bash
# run on node-persist
sudo mkdir -p /mnt/object
sudo chown -R cc /mnt/object
sudo chgrp -R cc /mnt/object

# run on node-persist
rclone mount chi_tacc:object-persist-project31 /mnt/object --read-only --allow-other --daemon

ls /mnt/object
```
To confirm that it's there

### Option 2: Create Volume
First, create the volume:

``` bash
# runs on node
docker volume create speech_commands
```

Then, to populate it with data, run

``` bash
# runs on node
docker compose -f docker/docker-compose-data.yaml up -d
```

This will run a temporary container that downloads the speech command dataset, organizes it in the volume, and then stops. It may take a minute or two. You can verify with

``` bash
# runs on node
docker ps
```

that it is done - when there are no running containers.

Finally, verify that the data looks as it should. Start a shell in a temporary container with this volume attached, and `ls` the contents of the volume:

``` bash
# runs on node
docker run --rm -it -v speech_commands:/mnt alpine ls -l /mnt/speech_commands/
```

it should show “evaluation”, “validation”, and “training” subfolders.

### Option 3: Manually download and orgainze
